In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers
from keras.models import Model
import keras
from datetime import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense, Input
from scripts.modelling import transform, invert_transform, root_mean_squared_error, mean_absolute_error, model_wrap

def split_data(data, window_size=15):
    x = []
    y = []
    start = 0
    end = start + window_size

    while end < data.shape[0]:
        x.append(data.iloc[start:end, 0:4])
        y.append(data.iloc[end, [0, 3]])

        start = end
        end = start + window_size

    x = np.array(x)
    y = np.array(y)

    split = int(x.shape[0]*0.7)

    return x[0:split, :, :], y[0:split], x[split:, :, :], y[split:]

def NN(inp_len: int):

    input_layer = Input(shape=(inp_len, 4))

    # First LSTM layer
    x = LSTM(units=50, return_sequences=True)(input_layer)
    x = Dropout(0.2)(x)

    # Second LSTM layer
    x = LSTM(units=50, return_sequences=True)(x)
    x = Dropout(0.2)(x)

    # Third LSTM layer
    x = LSTM(units=50, return_sequences=True)(x)
    x = Dropout(0.2)(x)

    # Fourth LSTM layer
    x = LSTM(units=50)(x)
    x = Dropout(0.2)(x)

    # Output layer
    output_layer = Dense(units=2)(x)

    # Define the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [3]:
from polygon import RESTClient
from dotenv import load_dotenv
import os

load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")
client = RESTClient(api_key=api_key)

today = datetime.today().strftime('%Y-%m-%d')
last_year = (datetime.today() - pd.DateOffset(years=2)).strftime('%Y-%m-%d')

ticker = 'AMD'
resp = pd.DataFrame(client.get_aggs(ticker, multiplier=1, timespan='minute', from_=last_year, to=today, limit=120000))
resp = resp.drop(columns=['timestamp', 'otc'])
#resp.to_csv(f'{ticker}_{today}.csv', index=False)

In [ ]:
'''
news_articles = client.list_ticker_news(
	"AAPL", 
	params={"published_utc.gte": today}, 
	order="desc", 
	limit=1000
	)

for article in news_articles:
    print(f"{article.title} [Insights: {article.insights}]")
    '''

In [ ]:
data = pd.read_csv(f'AAPL_2024-12-17.csv')
x_train, y_train, x_test, y_test = split_data(data)
display(x_train.shape, y_test.shape)

In [ ]:
model = model_wrap(model=NN(x_train.shape[1]), transform=transform, invert_transform=invert_transform)
model.model.summary()

loss = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.fit(x_train, y_train, loss=loss, optimizer=optimizer, epochs=100)

In [ ]:
y_pred = model.predict(x_train, y_train)

res_df = pd.DataFrame({'Open MAE': mean_absolute_error(y_train[:, 0], y_pred[:, 0]),
                       'Close MAE': mean_absolute_error(y_train[:, 1], y_pred[:, 1])}, index=[0])

display(res_df)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].scatter(y_train[:, 0], y_pred[:, 0])
ax[1].scatter(y_train[:, 1], y_pred[:, 1])
ax[0].plot(y_train[:, 0], y_train[:, 0], c='r')
ax[1].plot(y_train[:, 1], y_train[:, 1], c='r')

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(y_train[:, 0])
ax[0].plot(y_pred[:, 0])
ax[1].plot(y_train[:, 1])
ax[1].plot(y_pred[:, 1])

display(y_pred[:, 0])

In [ ]:
y_pred = model.predict(x_test, y_test)
display(y_test.shape, y_pred.shape)

res_df = pd.DataFrame({'Open MAE': mean_absolute_error(y_test[:, 0], y_pred[:, 0]),
                       'Close MAE': mean_absolute_error(y_test[:, 1], y_pred[:, 1])}, index=[0])

display(res_df)

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].scatter(y_test[:, 0], y_pred[:, 0])
ax[1].scatter(y_test[:, 1], y_pred[:, 1])
ax[0].plot(y_test[:, 0], y_test[:, 0], c='r')
ax[1].plot(y_test[:, 1], y_test[:, 1], c='r')

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(y_test[:, 0])
ax[0].plot(y_pred[:, 0])
ax[1].plot(y_test[:, 1])
ax[1].plot(y_pred[:, 1])

display(y_pred[:, 0])